# Callback Config

The `callback` is an essential utility in `graphai` that allows us to easily stream output from various nodes in our graph. Through the `Callback` class we can modify what our returned tokens look like _and_ also what we are and are _not_ returning.

In [ ]:
!pip install -qU graphai-lib

## Default Callback

Let's start with the default callback, we initialize it like so:

In [1]:
from graphai.callback import Callback

cb = Callback()

We can setup a simple `stream` function to test this out:

In [2]:
async def stream(cb: Callback, text: str):
    tokens = text.split(" ")
    for token in tokens:
        await cb.acall(token)
    await cb.close()
    return

To stream we use `asyncio.create_task` to _create the task_ and then we can use `async for` to stream the tokens from our initialized callback object (`cb`):

In [3]:
import asyncio

response = asyncio.create_task(stream(cb, "Hello over there"))

test = []

async for token in cb.aiter():
    test.append(token)

test

['Hello', 'over', 'there', '<graphai:END:>']

When streaming we will see each token/word is output, with a final token to identify the end of the stream. By default the final token is `"<graphai:END:>"`, each part of this token is configurable and is split into three parts:

- `identifier`: the identifier of the callback, in this case `graphai`
- `token`: the token to be returned, in this case `END`
- `params`: the parameters to be returned, in this case an empty string

The default callback format here is:

```
<{identifier}:{token}:{params}>
```

We can modify the structure and the `identifier` as preferred, for example we can change the `identifier` to `demo` and the structure to `[{identifier}:{token}:{params}]` like so:

In [4]:
cb = Callback(identifier="demo", special_token_format="[{identifier}:{token}:{params}]")

Rerun the stream:

In [5]:
response = asyncio.create_task(stream(cb, "Hello over there"))

test = []

async for token in cb.aiter():
    test.append(token)

test

['Hello', 'over', 'there', '[demo:END:]']

We can also drop one or more of the components, if we'd like to keep only the `token` for example we can:

In [6]:
cb = Callback(identifier="demo", special_token_format="<<{token}>>")

In [7]:
response = asyncio.create_task(stream(cb, "Hello over there"))

test = []

async for token in cb.aiter():
    test.append(token)

test

['Hello', 'over', 'there', '<<END>>']

## Callback Across Nodes

Our callback inherits information about which node it is being executed from, this is useful when we may want to treat streamed output differently depending on it's source. Let's setup a simple graph with a few nodes to see this in action:

In [8]:
from graphai import Graph, node
import asyncio


@node(start=True)
async def node_start(input: str):
    await asyncio.sleep(0.1)
    print(">> node_start")
    # no stream added here
    return {"input": input}


@node(stream=True)
async def node_a(input: str, callback: Callback):
    await asyncio.sleep(0.1)
    print(">> node_a")
    tokens = ["Hello", "World", "!"]
    for token in tokens:
        await callback.acall(token)
    return {"input": input}


@node(stream=True)
async def node_b(input: str, callback: Callback):
    await asyncio.sleep(0.1)
    print(">> node_b")
    tokens = ["Here", "is", "node", "B", "!"]
    for token in tokens:
        await callback.acall(token)
    return {"input": input}


@node
async def node_c(input: str):
    await asyncio.sleep(0.1)
    print(">> node_c")
    # no stream added here
    return {"input": input}


@node(stream=True)
async def node_d(input: str, callback: Callback):
    await asyncio.sleep(0.1)
    print(">> node_d")
    tokens = ["Here", "is", "node", "D", "!"]
    for token in tokens:
        await callback.acall(token)
    return {"input": input}


@node(end=True)
async def node_end(input: str):
    await asyncio.sleep(0.1)
    print(">> node_end")
    return {"input": input}


graph = Graph()

nodes = [node_start, node_a, node_b, node_c, node_d, node_end]

for i, node_fn in enumerate(nodes):
    graph.add_node(node_fn)
    if i > 0:
        print(f"Adding edge from {nodes[i - 1].__name__} to {node_fn.__name__}")
        graph.add_edge(nodes[i - 1], node_fn)

graph.compile()

Adding edge from node_start to node_a
Adding edge from node_a to node_b
Adding edge from node_b to node_c
Adding edge from node_c to node_d
Adding edge from node_d to node_end


Let's see this in action, note that we've added a `print` statement to each node to see the order in which they are executed, which we expect to align with the `{token}` field being output by node `start` and `end` special tokens.

In [9]:
cb = graph.get_callback()

asyncio.create_task(graph.execute(input={"input": "Hello"}, callback=cb))

async for token in cb.aiter():
    print(token)

>> node_start
<graphai:node_a:start:>
<graphai:node_a:>
>> node_a
Hello
World
!
<graphai:node_a:end:>
<graphai:node_b:start:>
<graphai:node_b:>
>> node_b
Here
is
node
B
!
<graphai:node_b:end:>
>> node_c
<graphai:node_d:start:>
<graphai:node_d:>
>> node_d
Here
is
node
D
!
<graphai:node_d:end:>
>> node_end
<graphai:END:>


We can see everything being output here, with each node we automatically output a `start` and `end` special token. Between these special tokens we know that every chunk of information being streamed back to us is being executed from inside that node.

This behaviour can be particularly useful when wanted to treat output from various nodes differently, for example we may want to render that a specific tool is being used inside a research agent application, before rendering the research agent's final response as we would typically render streamed tokens.

### Custom Callback

We can also modify the callback tokens as we did before, for example we can change the `identifier` to `demo` and the structure to `[{identifier}:{token}:{params}]` like so:

In [10]:
cb = Callback(identifier="demo", special_token_format="[{identifier}:{token}:{params}]")

We now execute the graph with the custom callback handler:

In [11]:
asyncio.create_task(graph.execute(input={"input": "Hello"}, callback=cb))

async for token in cb.aiter():
    print(token)

>> node_start
[demo:node_a:start:]
[demo:node_a:]
>> node_a
Hello
World
!
[demo:node_a:end:]
[demo:node_b:start:]
[demo:node_b:]
>> node_b
Here
is
node
B
!
[demo:node_b:end:]
>> node_c
[demo:node_d:start:]
[demo:node_d:]
>> node_d
Here
is
node
D
!
[demo:node_d:end:]
>> node_end
[demo:END:]


With that we have our custom callback being used across our graph.

---